In [ ]:
!nvidia-smi -L

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ==== Install Dependencies
!pip install -q efficientnet-pytorch
!pip install -q albumentations
!pip install -U -q pytorch-fanatics 
!pip install -q pytorch_ranger

In [ ]:
# ==== Import Libraries

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns
import random
import os


import albumentations as aug
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt


from tqdm import tqdm

from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_fanatics.dataloader import Cloader
from pytorch_fanatics.utils import EarlyStop ,LRFinder 
from pytorch_fanatics.trainer import Trainer
from pytorch_fanatics.logger import Logger

import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pytorch_ranger import Ranger

from efficientnet_pytorch import EfficientNet
from pathlib import Path

from torchvision import transforms

In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

In [ ]:
# meta=pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

# Data Exploration

In [ ]:
def preprocessing(meta):
  meta['age_approx']=meta['age_approx']/90
  meta['sex']=meta['sex'].map({'male':1,'female':0})
  meta['sex'].fillna(-1,inplace=True)
  meta['anatom_site_general'].fillna('unknown',inplace=True)
  dummies=pd.get_dummies(meta['anatom_site_general'])
  meta.drop(columns=['anatom_site_general','lesion_id'],inplace=True)
  meta=pd.concat([meta,dummies],axis=1)
  return meta

In [29]:
!unzip -qq '/content/drive/MyDrive/siic-isic-224x224-images.zip'

In [30]:
df1=pd.read_csv('/content/ISIC_2020_Training_GroundTruth (1).csv')
df1

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


In [31]:
df1['diagnosis'].value_counts()

unknown                               27124
nevus                                  5193
melanoma                                584
seborrheic keratosis                    135
lentigo NOS                              44
lichenoid keratosis                      37
solar lentigo                             7
cafe-au-lait macule                       1
atypical melanocytic proliferation        1
Name: diagnosis, dtype: int64

In [32]:
df=df1

In [ ]:
!mkdir "/content/drive/MyDrive/ACMCancer/"

In [33]:
save_root = "/content/drive/MyDrive/ACMCancer/"

In [35]:
# training_data_path = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

X_train , X_val ,Y_train , Y_val = tts(df, df.target.values, test_size=0.25
                                       ,random_state=42,stratify=df.target.values)
X_train       = X_train.reset_index(drop=True)
X_val         = X_val.reset_index(drop=True)

In [48]:
# ===== Augmentations

mean       = (0.485, 0.456, 0.406)
std        = (0.229, 0.224, 0.225)
image_size=224
train_tfms = aug.Compose([
            aug.RandomSizedCrop(min_max_height=(64,224),height=224,width=224,p=0.5),
            # aug.Resize(256,256),
            aug.HorizontalFlip(p=0.5),
            aug.RandomBrightnessContrast(0.1,0.1),
            #aug.HueSaturationValue(10,10,10),
            aug.RGBShift(),
            aug.RandomContrast(limit=0.2),
            aug.RandomGamma(),
            #aug.ShiftScaleRotate(rotate_limit=(-45,45)),
            aug.GaussNoise(p=0.35),
            aug.IAASharpen(),
            aug.ToGray(p=0.35),
            aug.CLAHE(clip_limit=4.0, p=0.7),
            #aug.RandomSizedCrop(min_max_height=(64,256),height=256,width=256,p=0.5),
            aug.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
            aug.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            #aug.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

test_tfms  = aug.Compose([
            aug.Resize(256,256),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

In [ ]:
!pip install -U git+https://github.com/ildoonet/cutmix
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

In [37]:
training_data_path='/content/train'

In [49]:
train_images     = X_train.image_name.values.tolist()
train_images     = [os.path.join(training_data_path, i+".png") for i in train_images]

test_images      = X_val.image_name.values.tolist()
test_images      = [os.path.join(training_data_path, i+".png") for i in test_images]

train_dataset    = Cloader(train_images,X_train.target.values,None,train_tfms)
#train_dataset    = CutMix(train_dataset, num_class=8, beta=1.0, prob=0.5, num_mix=2)
test_dataset     = Cloader(test_images,X_val.target.values,None,test_tfms)

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=2)
val_dataloader   = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=2)

device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [41]:
# ===== Define model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model=EfficientNet.from_pretrained('efficientnet-b0',num_classes=2)
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        out = self.base_model(image)
        targets = torch.tensor(targets,dtype=torch.int64)
        loss = nn.CrossEntropyLoss()(out.view(batch_size,2), targets)
        return out, loss

model = Net()
model.to(device);

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [42]:
def softmax(array):
    return np.exp(array)/np.sum(np.exp(array),axis=1).reshape(-1,1)

In [ ]:
model

In [44]:
optimizer = Ranger(model.parameters(),lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer,factor=0.8, mode="min", patience=2)

trainer   = Trainer(model=model,optimizer=optimizer,device=device,val_scheduler=scheduler)
logger    = Logger()

es        = EarlyStop(patience=5,mode="min") # mode = min to minimise loss

In [50]:
epochs = 30

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    es(val_loss,model,model_path ="/content/drive/MyDrive/ACMCancer/best.pth")
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if es.early_stop:
        break

+ ===== Epoch 1/30 ===== +


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f5fd4a85b38>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f5fd4a85b38>>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/u


Metric Validation score improved (inf --> 0.4241439377939378). Saving model!
train_loss 0.5681604183395241 val_loss 0.4241439377939378 
val accuracy_score 0.9758512436609514 
 
+ ===== Epoch 2/30 ===== +



Metric Validation score improved (0.4241439377939378 --> 0.15367427750213722). Saving model!
train_loss 0.274258891156511 val_loss 0.15367427750213722 
val accuracy_score 0.9822506640907993 
 
+ ===== Epoch 3/30 ===== +



Metric Validation score improved (0.15367427750213722 --> 0.09395606369211641). Saving model!
train_loss 0.12155551665212051 val_loss 0.09395606369211641 
val accuracy_score 0.9823714078724946 
 
+ ===== Epoch 4/30 ===== +


...


Metric Validation score improved (0.07127932655988223 --> 0.07052276564877062). Saving model!
train_loss 0.07544472778846473 val_loss 0.07052276564877062 
val accuracy_score 0.9823714078724946 
 
+ ===== Epoch 27/30 ===== +



EarlyStop count: 1 out of 5
train_loss 0.07535957841938573 val_loss 0.07073024227409751 
val accuracy_score 0.9823714078724946 
 
+ ===== Epoch 28/30 ===== +



EarlyStop count: 2 out of 5
train_loss 0.07616483848518009 val_loss 0.07096441833726691 
val accuracy_score 0.9823714078724946 
 
+ ===== Epoch 29/30 ===== +



EarlyStop count: 3 out of 5
train_loss 0.0750721341090101 val_loss 0.07144473314385966 
val accuracy_score 0.9823714078724946 
 
+ ===== Epoch 30/30 ===== +



EarlyStop count: 4 out of 5
train_loss 0.07510738525218409 val_loss 0.07106930511359945 
val accuracy_score 0.9823714078724946 
 


In [ ]:
torch.save(model.state_dict(),"/content/drive/MyDrive/ACMCancer/last.pth")